# COM-402: Password Cracking Demo

The objective of this exercise is for you to crack some passwords. The exercise consists of four parts. In each part, the setting is slightly different, modifying the way the cracking should be executed.

We do not provide a docker container for this session as no specific environment is required. Later, we will release a solution written in Python.


In [1]:
import hashlib
import itertools
import multiprocessing
import time
from itertools import repeat

from tqdm import tqdm

## Part 1: Brute force attack

In this part you should implement a brute-force attack. Passwords are randomly generated from the set of lowercase letters and digits (‘abcd...xyz0123...9’) and have length 4 or 5 characters. Generated passwords are then hashed with SHA-256 and corresponding hexdigests are sent to you in the file.

The list of SHA-256 digests you need to crack is:

- `19dbaf86488ec08ba7a824b33571ce427e318d14fc84d3d764bd21ecb29c34ca`
- `dd9ad1f17965325e4e5de2656152e8a5fce92b1c175947b485833cde0c824d64`
- `845e7c74bc1b5532fe05a1e682b9781e273498af73f401a099d324fa99121c99`
- `a6fb7de5b5e11b29bc232c5b5cd3044ca4b70f2cf421dc02b5798a7f68fc0523`
- `1035f3e1491315d6eaf53f7e9fecf3b81e00139df2720ae361868c609815039c`
  
What can you say about the computational time required? Can you parallelize this attack?

In [ ]:
end = False
def block_func(combination):
    """Take one combination, compute its hash, and return if a match is found"""
    if end: 
        return
    password = "".join(combination)
    h = hashlib.sha256(password.encode())
    digest = h.hexdigest()
    if digest in all_hashes:
        print("{} === {}".format(digest, password))
        return digest

# all the possible characters
charset = "abcdefghijklmnopqrstuvwxyz1234567890"

# the list of all the hashes
all_hashes = set([
    "19dbaf86488ec08ba7a824b33571ce427e318d14fc84d3d764bd21ecb29c34ca",
    "dd9ad1f17965325e4e5de2656152e8a5fce92b1c175947b485833cde0c824d64",
    "845e7c74bc1b5532fe05a1e682b9781e273498af73f401a099d324fa99121c99",
    "a6fb7de5b5e11b29bc232c5b5cd3044ca4b70f2cf421dc02b5798a7f68fc0523",
    "1035f3e1491315d6eaf53f7e9fecf3b81e00139df2720ae361868c609815039c"
    ])
hashes_to_crack = len(all_hashes)


begin = time.time()

# create a pool of processes
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count()) #TODO: try with a single thread (processes=1) instead, What do you observe?
total_matches = []

for length in range(4, 6):
    print("Trying passwords of length", length)
    
    # TODO: create a generator for the set of passwords for this length
    # HINT: use itertools
    combinations_generator = itertools.product(charset, repeat=length)

    # compute number of possible passwords
    total_combinations = len(charset)**length
    print("Total combinations:", total_combinations)

    if len(all_hashes) == len(total_matches):
        break
        
    # TODO: iterate through the combinations, leveraging multithreading with the pool
    # HINT: use pool.imap_unordered(...)
    for i, match in enumerate(pool.imap_unordered(block_func, combinations_generator, 1000)):
        if match is not None:
            total_matches.append(match)
            if len(all_hashes) == len(total_matches):
                break
        
        if i % 100 == 0:
            print("Progress for length {}: {:.3f}%".format(length,100*i/total_combinations), end="\r")
        

end = time.time()
print("Time for naïve: {:.3f}s".format(end-begin))

Trying passwords of length 4
Total combinations: 1679616


## Part 2: Dictionary attack with rules

In the previous part, you implemented the brute force attack and faced one of its drawbacks. Unfortunately, people very rarely use random passwords. Instead, they use some common words and sometimes modify them slightly. This is a fortunate fact for password crackers, because they can use ‘dictionary attacks’ to crack the passwords more efficiently than with brute-force. In this part you should implement one such dictionary attack. We generate a password by selecting a word from a large dictionary, [`rockyou.txt`](https://github.com/brannondorsey/naive-hashcat/releases/download/data/rockyou.txt), and then randomly applying some of the common user modifications:

- capitalize the first letter and every letter which comes after a digit (for example: ‘com402class’ becomes ‘Com402Class’). If you are using Python, this is easily achieved by ‘title()’ function from string module (‘com402class’.title() will give you ‘Com402Class’)
- change ‘e’ to ‘3’
- change ‘o’ to ‘0’ (that’s small letter ‘o’ to zero)
- change ‘i’ to ‘1’

Note that those operations are not all commutative.
(For instance, ‘window’ can become ‘W1ndow’, or ‘w1nd0w’, or W1Ndow,...)

The list of SHA-256 digests you need to crack is:

- `3d43987eeb0e001390791134ea47511a9758eaba9e07d61bcfae76323cdc9d14`
- `824aea9643be485d330860886e41599e26e190dd4c6eee203c80f1247ea5457b`
- `002ff8b4fb4538e0a44a374e45898e7140e24ef2be7814ccd71eafce946db60e`

**Note:** the file encoding or `rockyou.txt` is `latin-1`, so you should open the dictionary using:
```python
open("rockyou.txt",encoding="latin-1")
```

What do you observe compared to part 1?

In [ ]:
all_hashes = set(["3d43987eeb0e001390791134ea47511a9758eaba9e07d61bcfae76323cdc9d14",
                  "824aea9643be485d330860886e41599e26e190dd4c6eee203c80f1247ea5457b",
                  "002ff8b4fb4538e0a44a374e45898e7140e24ef2be7814ccd71eafce946db60e"])
def modif1(p: str):
    return p.title()
def modif2(p: str):
    return p.replace("e", "3")
def modif3(p: str):
    return p.replace("o", "0")
def modif4(p: str):
    return p.replace("i", "1")

def modif_and_hash(p):
    """Try all modifications and hashes from one given base password."""
    if len(all_hashes) == 0:
        return
    p = p.replace("\n", "") # ensure there are no \n at the end
    # create the alternative from the base passwords
    all_versions = set([p]) # include the base password
    for comb in all_modifs_combinations:
        p_temp = p
        for modificator in comb:
            p_temp = modificator(p_temp)
        all_versions.add(p_temp)
    # Hash and compare each of them
    for version in all_versions:
        hash = hashlib.sha256(version.encode()).hexdigest()
        if hash in all_hashes:
            print("{} === {} (from {})".format(hash, version, p))
            all_hashes.remove(hash)
            return p, hash

# generating all possible combinations of password modifications
all_modifs_combinations = set()
all_modifs = [modif1, modif2, modif3, modif4]
for length in range(1, len(all_modifs)+1):
    for comb in itertools.permutations(all_modifs, length):
        all_modifs_combinations.add(comb)
    
file = open("rockyou.txt", encoding="latin-1")
pool = multiprocessing.Pool(multiprocessing.cpu_count()) # define a pool of workers
results = []

# read the file by chunks of 10k rows at a time, then feed one
# rows one after the other to a worker
for r in pool.imap_unordered(modif_and_hash, file, 10000):
    if r is not None:
        print(r)
        results.append(r)

file.close()

## Part 3: Dictionary attack with salt

In the previous part of the exercise you implemented a dictionary attack. You should notice that once you have a dictionary you can compute the hashes of all those words in it, and create a lookup table. This way, each next password you want to crack is nothing more than a query in the lookup table. To tackle this problem, passwords are usually ‘salted’ before hashing. Salt is exactly two characters long and it contains only hexadecimal characters. In this part of the exercise you should implement another attack using a dictionary. We generate a password by simply selecting a random word from a dictionary ([rockyou.txt](https://github.com/brannondorsey/naive-hashcat/releases/download/data/rockyou.txt)) and appending a random salt to it. The password is then hashed with SHA-256 and hexdigest and salt are available to you. Your task is to crack the passwords.

The list of SHA-256 digests you need to crack and the salt (in brackets) used are:

- `69839ca8ae00839a988b8e0260091d15425df1265becd4548763008284a2ea50` (`b9`)
- `ce85f06a4ddf38633149a9098cdd1d672f61c109a4348ca116441ddaa2129b9e` (`be`)
- `858930e3415097b129fdd54dda2032fffb378a4d07eaf1fce69158890bb6a242` (`72`)


Why is it a good idea to salt the passwords? Estimate the complexity required in this part if the salts were not provided. What additional security countermeasures could you think of?


**Note1:** The SHA-256 digest of a password 'psswd' is the result of SHA-256(psswd). The digest of a salted password with salt XX is SHA-256(psswdXX).

**Note2:** Not all dictionaries are the same, be aware that if you implement the attack correctly but you can’t crack the passwords, then you might be using a dictionary which doesn’t contain all the words as the dictionary we used.

**Note3:** In order to check the passwords, just compute the hash of the word (with or without salt):

- macOS cli: `echo -n "psswd" | shasum -a 256`
- unix cli: `echo -n "psswd" | shasum -a 256`
- Windows: look into _Microsoft File Checksum Integrity Verifier_ or use an online hasher

**Note4:** The attacks you implement might take some time when you run them. Depending on your hardware and your implementation, the attacks may run for more than 30 minutes.

In [ ]:
all_hashes = set(["69839ca8ae00839a988b8e0260091d15425df1265becd4548763008284a2ea50",
                  "ce85f06a4ddf38633149a9098cdd1d672f61c109a4348ca116441ddaa2129b9e",
                  "858930e3415097b129fdd54dda2032fffb378a4d07eaf1fce69158890bb6a242"])

def salt_and_hash(p):
    """Take one password, and hash it using all the possible salts."""
    salts = ["b9", "be", "72"]
    p = p.replace("\n", "") # remove possible trailing \n
    for s in salts:
        salted = p+s
        hash = hashlib.sha256(salted.encode()).hexdigest()
        if hash in all_hashes:
            print("{} === {} (salt {})".format(hash, p, s))
            return p, hash
        

file = open("rockyou.txt", encoding="latin-1")
pool = multiprocessing.Pool(multiprocessing.cpu_count()) # define a pool of workers
results = []

# read the file by chunks of 10k rows at a time, then feed one
# rows one after the other to a worker
for r in pool.imap_unordered(salt_and_hash, file, 10000):
    if r is not None:
        print(r)
        results.append(r)

file.close()

## Part 4: A CTF challenge

In this part, you will solve a CTF challenge. The challenge creates a password by selecting a random word from a dictionary ("rockyou") and appending a random salt to it. The password is then hashed with multiple hash algorithms for 32 iterations and the result is available to you. Your task is to crack the password with salt.

The total lenth of the password and salt is 128 bytes, which makes it infeasible to crack with brute force or rainbow tables. A hint is to observe the hash calculation algorithm carefully and think if there is any flaw in it.

The challenge script is attached as `ex4.py`. After downloading the "rockyou" dictionary from [here](http://downloads.skullsecurity.org/passwords/rockyou-withcount.txt.bz2), you can run the script on your machine and write a client script to interact with it. Here is the script template, you can complete it with your password cracking solution:

```python
import os
import hashlib
import socket
import threading
import socketserver
import struct
import time
import threading

from base64 import b64encode, b64decode
from pwn import *

def md5(bytestring):
    return hashlib.md5(bytestring).digest()
def sha(bytestring):
    return hashlib.sha1(bytestring).digest()
def blake(bytestring):
    return hashlib.blake2b(bytestring).digest()
def scrypt(bytestring):
    l = int(len(bytestring) / 2)
    salt = bytestring[:l]
    p = bytestring[l:]
    return hashlib.scrypt(p, salt=salt, n=2**16, r=8, p=1, maxmem=67111936)

def xor(s1, s2):
    return b''.join([bytes([s1[i] ^ s2[i % len(s2)]]) for i in range(len(s1))])

def main():
    io = remote('127.0.0.1', 1337)
    print(io.recv(1000))
    ans_array = bytearray()
    while True:
        buf = io.recv(1)
        if buf:
            ans_array.extend(buf)
        if buf == b'!':
            break

    password_hash_base64 = ans_array[ans_array.find(b"b'") + 2: ans_array.find(b"'\n")]
    password_hash = b64decode(password_hash_base64)
    print('password:', password_hash)
    method_bytes = ans_array[
        ans_array.find(b'used:\n') + 6 : ans_array.find(b'\nYour')
    ]
    methods = method_bytes.split(b'\n')
    methods = [bytes(x.strip(b'- ')).decode() for x in methods]
    print(methods)

    # TODO: Add your password cracking code here
    # Note that the password is wrapped with a '{}'
    password = b'{YOUR_PASSWORD}'

    io.send(password)
    io.interactive()

if __name__ == '__main__':
    main()
```